In [1]:
import requests
import json
import time
import re
import datetime
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import yaml
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.neighbors import NearestNeighbors
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
import os, sys, glob
import kaleido
from PIL import Image
from fpdf import FPDF

In [2]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_weekday ,var_weekday_name

In [3]:
def eda(data):
    output=[]
    for col in data.columns:
        duplicatedvalue = data[col].duplicated().sum()
        duplicatedrows = data.duplicated().sum()
        missingvalue = np.sum(pd.isna(data[col]))
        uniquevalue = data[col].nunique()
        datatype = str(data[col].dtype)
        
        output.append([col, duplicatedvalue, duplicatedrows, missingvalue, uniquevalue, datatype])
        
    output = pd.DataFrame(output) 
    output.columns = ['Features', 'Duplicated Values', 'Duplicated Rows', 'Missing Values', 'Unique Values', 'Data Type']
    display(output)

In [4]:
def perc_on_bar(plot, feature):
    total = len(feature)
    for p in ax.patches:
        percentage = "{:.1f}%".format(100 * p.get_height() / total)
        x = p.get_x() + p.get_width() / 2 - 0.05
        y = p.get_y() + p.get_height()
        ax.annotate(percentage, (x, y), size=12)
    plt.show()

In [5]:
def viz(data, indice):
    fig = go.Figure(data=[go.Candlestick(x=data['UTC_Time'],
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'])])

    fig.update_layout(xaxis_rangeslider_visible=False,
                      title="Neighbor: " + indice + " " + pair + " " + timeframe,
                      title_font_color="blue",
                      title_font_size = 20)
    
    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])])
    
    fig.write_image(path + "/" + indice + "_chart.png")
    
    fig.show()

In [6]:
def market_order(instrument, units, take_profit, stop_loss):
    login_request_body = {
        "order": {
            "type": "MARKET",
            "instrument": instrument,
            "units": units,
            "timeInForce": "IOC",
            "positionFill": "DEFAULT",
            "takeProfitOnFill": {
                "price": take_profit
            },
            "stopLossOnFill": {
                "price": stop_loss
            }
        }
    }
    response = requests.post(provider_api_url, data=json.dumps(login_request_body),
                             headers=request_headers,
                             verify=False)
    response
    response.status_code

### The Configs for Run:

In [7]:
with open ('config.yml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    oanda_api_key = cfg['creds']['oanda_api']
    account_number = cfg['creds']['account_number'] 

### <font color='red'>Currency Pair</font>

In [8]:
Load_10K_Records=True

currency_pairs = ['EUR_USD','EUR_GBP','EUR_NZD','EUR_AUD','EUR_CHF','EUR_CAD',
                  'GBP_USD','GBP_CHF','GBP_NZD','GBP_AUD','GBP_CAD','AUD_USD',
                  'AUD_CAD','AUD_CHF','AUD_NZD','NZD_USD','NZD_CHF','NZD_CAD',
                  'USD_CAD','USD_CHF','CAD_CHF']

currency_pairs = ["EUR_GBP"]


timeframe = "H4"
#D #H1 #H4 M30
# https://developer.oanda.com/rest-live-v20/instrument-df/#CandlestickGranularity
price_char = "M"
#M(midpoint candles) #B(bid candles) #A(ask candles) #BA
price_com = "mid"
#mid #bid #ask

# def of OANDA request variable
provider_api_url = 'https://api-fxpractice.oanda.com/v3/accounts/101-001-16477519-001/orders'
request_headers = {
    "Authorization": oanda_api_key,
    "Accept-Datetime-Format": "RFC3339",
    "Connection": "Keep-Alive",
    "Content-Type": "application/json;charset=UTF-8"
}


In [9]:
provider_authorization = 'Bearer {0}'.format(oanda_api_key)

headers = {
    'Content-Type': 'application/json',
    'Authorization': provider_authorization,
}

### Logging

In [10]:
now = datetime.now()
for pair in currency_pairs:
    Log_Folder = now.strftime("%d-%m-%Y_%I-%M_%p")
    path = os.path.join(Log_Folder+"_"+pair)
    os.mkdir(path)

In [11]:
path

'01-10-2021_11-22_PM_EUR_GBP'

# Get Candlesticks Data

### Pricing & Spread Cost

In [12]:
for pair in currency_pairs:
    pricing_params = (
        ('instruments', pair),
    )
    response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/pricing'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()

In [13]:
time = response['time']
ask = response['prices'][0]['closeoutAsk']
bid = response['prices'][0]['closeoutBid']
print ('Date:', time, 'Ask:', ask, 'Bid:', bid)

Date: 2021-10-01T21:00:18.665716462Z Ask: 0.85629 Bid: 0.85558


### Open Positions

In [14]:
response = requests.get('https://api-fxpractice.oanda.com/v3/accounts/{}/openPositions'.format(account_number),
                            headers=headers,
                            params=pricing_params).json()
response['positions']

[{'instrument': 'EUR_USD',
  'long': {'units': '0',
   'pl': '-43414.1846',
   'resettablePL': '-43414.1846',
   'financing': '-410.1721',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'unrealizedPL': '0.0000'},
  'short': {'units': '-1000000',
   'averagePrice': '1.15667',
   'pl': '7208.9485',
   'resettablePL': '7208.9485',
   'financing': '-80.2932',
   'dividendAdjustment': '0.0000',
   'guaranteedExecutionFees': '0.0000',
   'tradeIDs': ['6177'],
   'unrealizedPL': '-2700.0000'},
  'pl': '-36205.2361',
  'resettablePL': '-36205.2361',
  'financing': '-490.4653',
  'commission': '2800.0400',
  'dividendAdjustment': '0.0000',
  'guaranteedExecutionFees': '0.0000',
  'unrealizedPL': '-2700.0000',
  'marginUsed': '23186.0000'}]

###  Candlestick Data

In [15]:
params_count = (
    ('price', price_char),
    ('count', '5000'),
    ('granularity', timeframe),
)


In [16]:
for pair in currency_pairs:
    first_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair), 
                            headers=headers,
                            params=params_count).json()

In [17]:
if Load_10K_Records:
    datetime_object = parser.parse(first_response['candles'][0]['time'])
    date= datetime_object - relativedelta(years=3)  
    from_date = date.replace(tzinfo=timezone.utc).timestamp()
    params_date = (
        ('count', '5000'),
        ('price', price_char),
        ('from', from_date),
        ('granularity', timeframe),)

    second_response = requests.get('https://api-fxpractice.oanda.com/v3/instruments/{}/candles'.format(pair),
                                   headers=headers,
                                   params=params_date).json()
            
    first_response= first_response['candles']  
    second_response= second_response['candles']
    second_response.extend(first_response)
    response=second_response
else:
    response=first_response['candles']

In [18]:
filename = "{}_{}.csv".format(pair, timeframe)
output = []
all_candlesticks = response

for i in range (len(all_candlesticks)):
    result= (convert_date(response[i]['time']))
    output.append([(result[0]),(result[1]),(result[2]),(result[3]),
                    response[i]['time'],
                    response[i]['volume'], 
                    response[i][price_com]['o'],
                    response[i][price_com]['h'],
                    response[i][price_com]['l'],
                    response[i][price_com]['c']])
    
output = pd.DataFrame(output)
output.columns = ['Date', 'Time', 'Weekday','Weekday_Name','UTC_Time', 'Volume', 'Open', 'High', 'Low', 'Close']
data = output.to_csv(filename, header = True, index = False)
data = pd.read_csv(filename)

In [19]:
data.head()

,Date,Time,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
0,2015-07-17,01:00:00,4,Friday,2015-07-17T01:00:00.000000000Z,920,0.69718,0.69758,0.69648,0.69693
1,2015-07-17,05:00:00,4,Friday,2015-07-17T05:00:00.000000000Z,3198,0.69696,0.69712,0.69444,0.69602
2,2015-07-17,09:00:00,4,Friday,2015-07-17T09:00:00.000000000Z,3671,0.69600,0.69895,0.69553,0.69728
3,2015-07-17,13:00:00,4,Friday,2015-07-17T13:00:00.000000000Z,3694,0.69730,0.69765,0.69364,0.69474
4,2015-07-17,17:00:00,4,Friday,2015-07-17T17:00:00.000000000Z,1204,0.69472,0.69526,0.69381,0.69419


In [20]:
data.tail()

,Date,Time,Weekday,Weekday_Name,UTC_Time,Volume,Open,High,Low,Close
9995,2021-10-01,01:00:00,4,Friday,2021-10-01T01:00:00.000000000Z,1344,0.85960,0.86062,0.85954,0.86042
9996,2021-10-01,05:00:00,4,Friday,2021-10-01T05:00:00.000000000Z,4841,0.86038,0.86240,0.85888,0.85932
9997,2021-10-01,09:00:00,4,Friday,2021-10-01T09:00:00.000000000Z,4968,0.85930,0.85932,0.85632,0.85684
9998,2021-10-01,13:00:00,4,Friday,2021-10-01T13:00:00.000000000Z,5967,0.85682,0.85694,0.85444,0.85517
9999,2021-10-01,17:00:00,4,Friday,2021-10-01T17:00:00.000000000Z,1206,0.85514,0.85606,0.85496,0.85594


In [21]:
data = data.drop_duplicates()

In [22]:
data = data.to_csv(filename, header = True, index = False)

In [23]:
data = pd.read_csv(filename)

In [24]:
data.shape

(9659, 10)

# <font color='red'>CANDLE INDEX NUMBER</font>

In [25]:
candle_no = len(data) - 2
candle_no

9657

# Feature Engineering

### CandleStick Data

In [26]:
data['O-H'] = data['Open'] - data['High']
data['O-L'] = data['Open'] - data['Low']
data['O-C'] = data['Open'] - data['Close']
data['H-L'] = data['High'] - data['Low']
data['H-C'] = data['High'] - data['Close']
data['L-C'] = data['Low'] - data['Close']

In [37]:
data['Direction'] = data['O-C'].apply(lambda x: 1 if x<0 else 0)

data['col_1'] = data['Open'] - data['Close']

for col1 in data['col_1']:
    print(name)
    
if data[data['col_1']] > 0:
    data['col_2'] = data['High'] - data['Open']
    data['col_3'] = data['Close'] - data['Low']
else:
    data['col_2'] = data['High'] - data['Close']
    data['col_3'] = data['Open'] - data['Low']


SyntaxError: unexpected EOF while parsing (<ipython-input-37-a630877ba4ad>, line 8)

In [ ]:
data = data.to_csv(filename, header = True, index = False)

In [ ]:
data = pd.read_csv(filename)

In [ ]:
data.head()

# Modeling

In [ ]:
data = pd.read_csv(filename)

In [ ]:
data = data.drop(columns=['Volume','Weekday','Date','Time',
                          'Weekday_Name','UTC_Time','Direction',
                          'Open', 'High', 'Low', 'Close'])

#'Volume', 'Direction'

### Scaling using Standard Scaler

In [ ]:
def find_k_similar_candles(candle_id, dataset, k=6):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = 'euclidean', algorithm = 'brute') 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i==0:
            display (pd.DataFrame(data.iloc[candle_id]).transpose())
            #print("Recommendation for {0}:\n".format(eurusd_data.index[candle_id]))
        else:
            #print("{0}: {1}, with distance of {2}".format(i,
            #                                               dataset.index[indices.flatten()[i]],
            #                                               distances.flatten()[i]))
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
                            dataset.iloc[indices.flatten()[i]]['O-H'],
                            dataset.iloc[indices.flatten()[i]]['O-L'],
                            dataset.iloc[indices.flatten()[i]]['O-C'],
                            dataset.iloc[indices.flatten()[i]]['H-L'],
                            dataset.iloc[indices.flatten()[i]]['H-C'],
                            dataset.iloc[indices.flatten()[i]]['L-C']])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance','O-H','O-L','O-C','H-L','H-C','L-C']
    display (output)
    
    return indices, distances

### Top 5 Similar Candlesticks

In [ ]:
indices, distances = find_k_similar_candles (candle_no,data)

In [ ]:
indices = indices[0:1][0]

In [ ]:
indices

### Currnet Market/Candlestick

In [ ]:
closed_candle = "currnet_market_data.csv"
data = pd.read_csv(filename)
data = data.iloc[candle_no-60:candle_no+1]
data.to_csv(path + "/" + closed_candle, header = True, index = False)
viz(data, "current_market")

### Recommendations

In [ ]:
for indice in indices[1:6]:
    recommendation_log = "{}_data.csv".format(indice)
    data = pd.read_csv(filename)
    data = data.iloc[indice:indice+50]
    data.to_csv(path + "/" + recommendation_log, header = True, index = False)
    print ('Neighbor:', indice, '|', '10K Records:', Load_10K_Records)
    viz(data, indice.astype(str))

In [ ]:
currentpath = os.path.join(sys.path[0])
pngfiles = []
pngfiles = glob.glob(currentpath+"/"+path+"/*.png")
pdf = FPDF()

for pngfile in pngfiles:
    pdf.add_page()
    pdf.image(pngfile, w=200, h=130)
    
pdf.output(path+"/recommendations.pdf", "F")